In [91]:
import pandas as pd
from tqdm import tqdm
import re
import numpy as np

In [2]:
df = pd.read_excel('00_DPF_클리닝_내역_원본.xlsx')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692017 entries, 0 to 692016
Data columns (total 48 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   클리닝진행상태    691904 non-null  object 
 1   클리닝기간구분    544758 non-null  object 
 2   차량번호       691873 non-null  object 
 3   차대번호       683210 non-null  object 
 4   구조변경일자     691859 non-null  object 
 5   현재차량등록번호   54 non-null      object 
 6   차명         691855 non-null  object 
 7   용도         691836 non-null  object 
 8   차종         688979 non-null  object 
 9   주행거리       691845 non-null  object 
 10  저감장치구분     686027 non-null  object 
 11  저감장치종류     691797 non-null  object 
 12  인증모델번호     691837 non-null  object 
 13  장치일련번호     688911 non-null  object 
 14  장치일련번호_전후  460072 non-null  object 
 15  클리닝일자      678012 non-null  object 
 16  클리닝종류      677939 non-null  object 
 17  제작사코드      54384 non-null   object 
 18  클리닝업체명     683078 non-null  object 
 19  접수지자체코드    54099 non-nu

In [11]:
df.tail()

,클리닝진행상태,클리닝기간구분,차량번호,차대번호,구조변경일자,현재차량등록번호,차명,용도,차종,주행거리,...,제작사서류첨부YN,클리닝지급일자,클리닝사업년도,소유주명,연락처,정산년도,정산일자,정산YN,정산기타,비고
692012,지급,특별,NaN,NaN,NaN,NaN,NaN,NaN,NaN,280602,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
692013,/BF-L00918,20180704,고정식,NaN,북항자동차정비검사소,NaN,20190201,150000,150000,보증기간내,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
692014,지급,정기,NaN,NaN,NaN,NaN,NaN,NaN,NaN,280602,...,Y,20180531,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
692015,지급,정기,NaN,NaN,NaN,NaN,NaN,NaN,NaN,180602,...,Y,20171024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
692016,지급,정기,NaN,NaN,NaN,NaN,NaN,NaN,NaN,134254,...,Y,20210107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df2 = pd.read_excel('220819_클리닝현황내역.xlsx')
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 757545 entries, 0 to 757544
Data columns (total 48 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   클리닝진행상태    757461 non-null  object 
 1   클리닝기간구분    601805 non-null  object 
 2   차량번호       757425 non-null  object 
 3   차대번호       748764 non-null  object 
 4   구조변경일자     757409 non-null  object 
 5   현재차량등록번호   63 non-null      object 
 6   차명         757405 non-null  object 
 7   용도         757381 non-null  object 
 8   차종         754525 non-null  object 
 9   주행거리       757396 non-null  object 
 10  저감장치구분     751574 non-null  object 
 11  저감장치종류     757337 non-null  object 
 12  인증모델번호     757383 non-null  object 
 13  장치일련번호     754389 non-null  object 
 14  장치일련번호_전후  525660 non-null  object 
 15  클리닝일자      743922 non-null  object 
 16  클리닝종류      743839 non-null  object 
 17  제작사코드      54393 non-null   object 
 18  클리닝업체명     748625 non-null  object 
 19  접수지자체코드    54106 non-nu

In [5]:
df.head()

,클리닝진행상태,클리닝기간구분,차량번호,차대번호,구조변경일자,현재차량등록번호,차명,용도,차종,주행거리,...,제작사서류첨부YN,클리닝지급일자,클리닝사업년도,소유주명,연락처,정산년도,정산일자,정산YN,정산기타,비고
0,지급,NaN,서울72더2071,KMJRL18SP4C002518,20041115,NaN,에어로버스,학교차량,대형승합,113669,...,Y,20160715,NaN,교육인적자원부(한국체육대학교),010-5330-2128,NaN,NaN,NaN,NaN,NaN
1,지급,NaN,서울72더2071,KMJRL18SP4C002518,20041115,NaN,에어로버스,학교차량,대형승합,110807,...,Y,20140826,NaN,교육인적자원부(한국체육대학교),010-5330-2128,NaN,NaN,NaN,NaN,NaN
2,지급,NaN,서울72거1699,KMJTG18BP1C002389,20041207,NaN,현대에어로시티540버스(터보),기타화물,대형승합,154807,...,Y,20140723,NaN,세기문화사,010-0000-0000,NaN,NaN,NaN,NaN,NaN
3,지급,NaN,경기72나9098,280224308491,20050113,NaN,신광치과진료차,기타승합,대형승합,122721,...,Y,20131230,NaN,경기도청,011-233-7893,NaN,NaN,NaN,NaN,NaN
4,지급,NaN,경기72나9098,280224308491,20050113,NaN,신광치과진료차,기타승합,대형승합,112315,...,Y,20121231,NaN,경기도청,011-233-7893,NaN,NaN,NaN,NaN,NaN


In [6]:
df2.head()

,클리닝진행상태,클리닝기간구분,차량번호,차대번호,구조변경일자,현재차량등록번호,차명,용도,차종,주행거리,...,제작사서류첨부YN,클리닝지급일자,클리닝사업년도,소유주명,연락처,정산년도,정산일자,정산YN,정산기타,비고
0,지급,NaN,서울72더2071,KMJRL18SP4C002518,20041115,NaN,에어로버스,학교차량,대형승합,113669,...,Y,20160715,NaN,교육인적자원부(한국체육대학교),010-5330-2128,NaN,NaN,NaN,NaN,NaN
1,지급,NaN,서울72더2071,KMJRL18SP4C002518,20041115,NaN,에어로버스,학교차량,대형승합,110807,...,Y,20140826,NaN,교육인적자원부(한국체육대학교),010-5330-2128,NaN,NaN,NaN,NaN,NaN
2,지급,NaN,서울72거1699,KMJTG18BP1C002389,20041207,NaN,현대에어로시티540버스(터보),기타화물,대형승합,154807,...,Y,20140723,NaN,세기문화사,010-0000-0000,NaN,NaN,NaN,NaN,NaN
3,지급,NaN,경기72나9098,280224308491,20050113,NaN,신광치과진료차,기타승합,대형승합,122721,...,Y,20131230,NaN,경기도청,011-233-7893,NaN,NaN,NaN,NaN,NaN
4,지급,NaN,경기72나9098,280224308491,20050113,NaN,신광치과진료차,기타승합,대형승합,112315,...,Y,20121231,NaN,경기도청,011-233-7893,NaN,NaN,NaN,NaN,NaN


In [7]:
total = pd.concat([df, df2], ignore_index=True)

In [21]:
total.shape

(1449562, 48)

In [ ]:
# total.groupby(list(total.columns)).groups

In [9]:
idx = [diff[0] for diff in total.groupby(list(total.columns)).groups.values() if len(diff) == 1]

In [10]:
# df는 index 692016 까지
# df2는 index 692017 부터

In [15]:
total.iloc[692012:692017]

,클리닝진행상태,클리닝기간구분,차량번호,차대번호,구조변경일자,현재차량등록번호,차명,용도,차종,주행거리,...,제작사서류첨부YN,클리닝지급일자,클리닝사업년도,소유주명,연락처,정산년도,정산일자,정산YN,정산기타,비고
692012,지급,특별,NaN,NaN,NaN,NaN,NaN,NaN,NaN,280602,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
692013,/BF-L00918,20180704,고정식,NaN,북항자동차정비검사소,NaN,20190201,150000,150000,보증기간내,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
692014,지급,정기,NaN,NaN,NaN,NaN,NaN,NaN,NaN,280602,...,Y,20180531,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
692015,지급,정기,NaN,NaN,NaN,NaN,NaN,NaN,NaN,180602,...,Y,20171024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
692016,지급,정기,NaN,NaN,NaN,NaN,NaN,NaN,NaN,134254,...,Y,20210107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
total.columns

Index(['클리닝진행상태', '클리닝기간구분', '차량번호', '차대번호', '구조변경일자', '현재차량등록번호', '차명', '용도',
       '차종', '주행거리', '저감장치구분', '저감장치종류', '인증모델번호', '장치일련번호', '장치일련번호_전후',
       '클리닝일자', '클리닝종류', '제작사코드', '클리닝업체명', '접수지자체코드', '보조금지급확정일자', '클리닝비용',
       '클리닝잔여보조금', '클리닝보증기간구분', '클리닝기록횟수경과', '전매연농도', '후매연농도', '전배기압력',
       '후배기압력', '접수시', '접수시도', '협회신청YN', '클리닝계획청구년월', '클리닝특별사유사유', '클리닝기타특별사유',
       '클리닝경정대상YN', '클리닝경정사유', '클리닝지급YN', '제작사서류첨부YN', '클리닝지급일자', '클리닝사업년도',
       '소유주명', '연락처', '정산년도', '정산일자', '정산YN', '정산기타', '비고'],
      dtype='object')

In [23]:
total[['저감장치구분', '저감장치종류', '인증모델번호', '장치일련번호', '장치일련번호_전후', '클리닝일자', '클리닝종류', '제작사코드', '클리닝업체명', '접수지자체코드']]

,저감장치구분,저감장치종류,인증모델번호,장치일련번호,장치일련번호_전후,클리닝일자,클리닝종류,제작사코드,클리닝업체명,접수지자체코드
0,1종,자연대형,4F1-MB-01,H3C005,NaN,20160408,이동식,NaN,동아진흥(화성),NaN
1,1종,자연대형,4F1-MB-01,H3C005,NaN,20140619,고정식,NaN,푸른하늘,NaN
2,1종,자연대형,4F1-MB-01,H3C008,NaN,20140502,고정식,NaN,명진공업사,NaN
3,1종,자연대형,4F1-SK-01,NaN,NaN,20131120,고정식,NaN,한국자동차대형검사소,NaN
4,1종,자연대형,4F1-SK-01,NaN,NaN,20121024,고정식,SK이노베이션(주),한국자동차대형검사소,경기 수원시청
...,...,...,...,...,...,...,...,...,...,...
1449557,NaN,NaN,NaN,NaN,BF-L00918,NaN,NaN,NaN,NaN,NaN
1449558,NaN,9.2,0.8,48,8,서울시,구로구,N,201808,배압상승
1449559,NaN,NaN,NaN,NaN,BF-L00918,20180404,고정식,NaN,(주)하나자동차공업사,NaN
1449560,NaN,NaN,NaN,NaN,NaN,20170728,고정식,NaN,북항자동차정비검사소,NaN


In [16]:
len(idx)

264878

In [17]:
idx[0]

1207312

In [18]:
diff1 = [one for one in idx if one <= 692016]
diff2 = [one for one in idx if one > 692016]

In [20]:
len(diff1), len(diff2)

(99661, 165217)

In [19]:
total.reindex(diff1)

,클리닝진행상태,클리닝기간구분,차량번호,차대번호,구조변경일자,현재차량등록번호,차명,용도,차종,주행거리,...,제작사서류첨부YN,클리닝지급일자,클리닝사업년도,소유주명,연락처,정산년도,정산일자,정산YN,정산기타,비고
500142,1톤,장거리운행차량,소형화물,172401,1종,복합소형,CF1-CR-01,H191014-130,1191201-897/211001-0060,20220228,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
640684,1톤,장거리운행차량,소형화물,181802,1종,복합소형,CF1-CR-01,H190905-277,191201-727/210301-819,20210426,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
640686,1톤,장거리운행차량,소형화물,181802,1종,복합소형,CF1-CR-01,H190905-277,200101-594/200429-563,20200616,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
444757,1톤,장거리운행차량,소형화물,223703,1종,복합소형,CF1-CR-01,H190511-112,200801-767/211130-1921,20220227,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
500405,(LIBERO),단거리운행차량,중형화물,314397,1종,복합소형,BF1-ED-01,XO100428,EDA26071,20201014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170140,청구,NaN,인천86자5606,KMFPA18TP5C005706,20130522,NaN,뉴파워트럭,장거리운행차량,대형화물,452771,...,Y,NaN,NaN,도명특송(주),011-9775-6767,NaN,NaN,NaN,NaN,NaN
50409,청구,NaN,인천86자9342,KL3K7CRF16K000207,20090323,NaN,대우16톤단축카고트럭,장거리운행차량,대형화물,199466,...,Y,NaN,NaN,조일통상(주),010-9342-7099,NaN,NaN,NaN,NaN,NaN
192577,청구,NaN,인천95바2746,KN90JFMND5GB3C002,20131015,NaN,한국특장기술22.5톤트럭,장거리운행차량,대형화물,286330,...,Y,NaN,NaN,(주)삼진상사,010-5310-3357,NaN,NaN,NaN,NaN,NaN
386663,청구,NaN,전북82사1961,KMFDA18TP7C020629,20181119,NaN,뉴파워트럭,장거리운행차량,대형화물,980208,...,Y,NaN,NaN,(유)탑로지스물류,010-7578-8100,NaN,NaN,NaN,NaN,NaN


In [31]:
total.columns

Index(['클리닝진행상태', '클리닝기간구분', '차량번호', '차대번호', '구조변경일자', '현재차량등록번호', '차명', '용도',
       '차종', '주행거리', '저감장치구분', '저감장치종류', '인증모델번호', '장치일련번호', '장치일련번호_전후',
       '클리닝일자', '클리닝종류', '제작사코드', '클리닝업체명', '접수지자체코드', '보조금지급확정일자', '클리닝비용',
       '클리닝잔여보조금', '클리닝보증기간구분', '클리닝기록횟수경과', '전매연농도', '후매연농도', '전배기압력',
       '후배기압력', '접수시', '접수시도', '협회신청YN', '클리닝계획청구년월', '클리닝특별사유사유', '클리닝기타특별사유',
       '클리닝경정대상YN', '클리닝경정사유', '클리닝지급YN', '제작사서류첨부YN', '클리닝지급일자', '클리닝사업년도',
       '소유주명', '연락처', '정산년도', '정산일자', '정산YN', '정산기타', '비고'],
      dtype='object')

In [33]:
total['클리닝진행상태'].unique()

array(['지급', '/H2M365', '/H2D019', ..., '(POTERII)', '-065', '/BF-L00918'],
      dtype=object)

In [36]:
temp = total['클리닝진행상태'].value_counts().to_frame()

In [38]:
temp

,클리닝진행상태
지급,1337005
신청,81288
청구,11658
승인,1865
미승인,64
...,...
/AE0528,2
/SF-1-0010829,2
/AI1827,2
/BN064,2


In [42]:
temp.iloc[5:]

,클리닝진행상태
1톤,19
(STAREX),14
/,14
(1%),10
9,10
...,...
/AE0528,2
/SF-1-0010829,2
/AI1827,2
/BN064,2


In [43]:
temp.iloc[5:].sum()

클리닝진행상태    17485
dtype: int64

In [52]:
total[total['클리닝진행상태'] == '9'].index

Int64Index([ 179574,  213064,  346713,  373372,  386221,  873455,  907369,
            1043378, 1071084, 1084461],
           dtype='int64')

In [50]:
total.iloc[179572:179578, :20]

,클리닝진행상태,클리닝기간구분,차량번호,차대번호,구조변경일자,현재차량등록번호,차명,용도,차종,주행거리,저감장치구분,저감장치종류,인증모델번호,장치일련번호,장치일련번호_전후,클리닝일자,클리닝종류,제작사코드,클리닝업체명,접수지자체코드
179572,지급,NaN,서울83머4161,KMFWVH7HP3U527202,20130801,NaN,스타렉스6밴오토(STAREX VAN),기타화물,중형화물,207146,1종,복합소형,BF1-CE-01,SF-1-005560,SF-1-011569/SF-1-012487,20210803,고정식,NaN,크린S,NaN
179573,지급,NaN,서울83머4161,KMFWVH7HP3U527202,20130801,NaN,스타렉스6밴오토(STAREX VAN),기타화물,중형화물,183818,1종,복합소형,BF1-CE-01,SF-1-005560,SF-1-010985/SF-1-011569,NaN,NaN,NaN,NaN,NaN
179574,9,20181122,고정식,NaN,주식회사 크린어스,NaN,20190621,150000,NaN,보증기간경과,0,7.0,0.2,100,0,NaN,NaN,N,201812,NaN
179575,지급,특별,서울83머4161,KMFWVH7HP3U527202,20130801,NaN,스타렉스6밴오토(STAREX VAN),기타화물,중형화물,167516,1종,복합소형,BF1-CE-01,SF-1-005560,NaN,20160729,고정식,NaN,주식회사 엘앤케이,NaN
179576,지급,정기,서울83머4161,KMFWVH7HP3U527202,20130801,NaN,스타렉스6밴오토(STAREX VAN),기타화물,중형화물,158327,1종,복합소형,BF1-CE-01,SF-1-005560,NaN,20151021,고정식,NaN,주식회사 엘앤케이,NaN
179577,지급,NaN,76조1640,KMJWWH7HP4U617047,20130731,NaN,스타렉스(STAREX),기타승합,중형승합,225215,1종,복합소형,BF1-CE-01,SF-1-005559,NaN,20170209,고정식,NaN,주식회사 엘앤케이,NaN


In [53]:
total.shape

(1449562, 48)

## df2 오류값 인덱스 확인
- 220819_클리닝현황내역.xlsx

In [54]:
df2.columns

Index(['클리닝진행상태', '클리닝기간구분', '차량번호', '차대번호', '구조변경일자', '현재차량등록번호', '차명', '용도',
       '차종', '주행거리', '저감장치구분', '저감장치종류', '인증모델번호', '장치일련번호', '장치일련번호_전후',
       '클리닝일자', '클리닝종류', '제작사코드', '클리닝업체명', '접수지자체코드', '보조금지급확정일자', '클리닝비용',
       '클리닝잔여보조금', '클리닝보증기간구분', '클리닝기록횟수경과', '전매연농도', '후매연농도', '전배기압력',
       '후배기압력', '접수시', '접수시도', '협회신청YN', '클리닝계획청구년월', '클리닝특별사유사유', '클리닝기타특별사유',
       '클리닝경정대상YN', '클리닝경정사유', '클리닝지급YN', '제작사서류첨부YN', '클리닝지급일자', '클리닝사업년도',
       '소유주명', '연락처', '정산년도', '정산일자', '정산YN', '정산기타', '비고'],
      dtype='object')

### 1\. 클리닝진행상태

In [56]:
df2['클리닝진행상태'].value_counts()

지급               699842
신청                40030
청구                 8801
미승인                  32
 1톤                  10
                  ...  
/SF-1-0010829         1
/AI1827               1
/BN064                1
/AE1546               1
/BF-L00918            1
Name: 클리닝진행상태, Length: 7912, dtype: int64

In [83]:
error1 = dict()
for one in tqdm(df2['클리닝진행상태'].value_counts().iloc[4:].index):
    temp = df2[df2['클리닝진행상태'] == one].index
    error1[one] = temp

100%|██████████| 7908/7908 [05:30<00:00, 23.91it/s]


### 2\. 클리닝기간구분

In [62]:
df2['클리닝기간구분'].value_counts()

정기                       363319
특별                       227762
기간산정                       1980
20180621                     65
20180620                     62
                          ...  
20180205                      1
20190329                      1
20191102                      1
20191221                      1
191201-615/210621-398         1
Name: 클리닝기간구분, Length: 662, dtype: int64

In [63]:
df2['클리닝기간구분'].value_counts().iloc[3:]

20180621                 65
20180620                 62
20180601                 60
20180615                 59
20180529                 59
                         ..
20180205                  1
20190329                  1
20191102                  1
20191221                  1
191201-615/210621-398     1
Name: 클리닝기간구분, Length: 659, dtype: int64

In [84]:
error2 = dict()
for one in tqdm(df2['클리닝기간구분'].value_counts().iloc[3:].index):
    temp = df2[df2['클리닝기간구분'] == one].index
    error2[one] = temp

100%|██████████| 659/659 [00:24<00:00, 26.57it/s]


### 3\. 차량번호

In [64]:
df2['차량번호'].value_counts()

고정식          7828
이동식           838
서울87아9432      16
중형화물           15
서울88아2986      15
             ... 
경기52러2477       1
02머3850         1
경북83아7024       1
09수8335         1
52서7113         1
Name: 차량번호, Length: 314631, dtype: int64

In [67]:
p3 = re.compile('[0-9]+')
uni3 = []
for one in df2['차량번호'].unique():
    try:
        if not p3.search(one):
            uni3.append(one)
    except:
        pass

In [68]:
uni3

['고정식', '이동식', '소형화물', '중형승용', '대형승용', '중형화물', '중형승합']

In [85]:
error3 = dict()
for one in tqdm(uni3):
    temp = df2[df2['차량번호'] == one].index
    error3[one] = temp

100%|██████████| 7/7 [00:00<00:00, 22.04it/s]


### 4\. 차대번호

In [69]:
df2['차대번호'].value_counts()

고정식                  16
KN96JCMSDXKB0U008    14
KMFLA19RPYC016851    14
KMFLA18AP5C007805    13
KN3LAP8Y22K018983    13
                     ..
KMJWWH7JP7U798125     1
KMHSD81VP4U666629     1
KNAJC52183A115585     1
KMHWP81HP2U411203     1
KMHSJ81WP7U160656     1
Name: 차대번호, Length: 316172, dtype: int64

In [70]:
p4 = re.compile('[0-9]+')
uni4 = []
for one in df2['차대번호'].unique():
    try:
        if not p4.search(one):
            uni4.append(one)
    except:
        pass

In [71]:
uni4

['고정식', '마이티', '포터Ⅱ슈퍼캡냉동탑차(PORTERⅡ)']

In [86]:
error4 = dict()
for one in tqdm(uni4):
    temp = df2[df2['차대번호'] == one].index
    error4[one] = temp

100%|██████████| 3/3 [00:00<00:00, 14.19it/s]


### 5\. 구조변경일자

In [72]:
df2['구조변경일자'].value_counts()

(주)서천제일자동차공업사    4135
20191127         1518
주식회사 크린어스        1502
20191126         1501
20191125         1495
                 ... 
20110806            1
20110813            1
20111228            1
중앙자동차정비공업사          1
20220422            1
Name: 구조변경일자, Length: 4438, dtype: int64

In [75]:
p5 = re.compile('[가-힣]+')
uni5 = []
for one in df2['구조변경일자'].unique():
    try:
        if p5.search(one):
            uni5.append(one)
    except:
        pass

In [76]:
uni5

['(주)서천제일자동차공업사',
 '(주)창신종합중기정비',
 '명진자동차',
 '(주)동방정비',
 '북항자동차정비검사소',
 '포천자동차검사소',
 '예지엠에스주식회사',
 '주식회사 오토종합정비',
 '주식회사 크린어스',
 '일진ECO',
 '(주)서천제일자동차공업사-이동식',
 '주식회사 엘앤케이',
 '크린ECO',
 '(주)부산정비',
 '(주)하나자동차공업사',
 '중앙자동차정비공업사',
 '하나자동차공업사-이동식',
 '주식회사 에코닉스',
 '해성',
 '(주)이엔드디(공장)',
 '통일로카독크',
 '1종',
 '단거리운행차량']

In [87]:
error5 = dict()
for one in tqdm(uni5):
    temp = df2[df2['구조변경일자'] == one].index
    error5[one] = temp

100%|██████████| 23/23 [00:00<00:00, 23.12it/s]


### 6\. 현재차량등록번호

In [77]:
df2['현재차량등록번호'].value_counts()

복합소형         42
세라컴(본사공장)    16
중형화물          5
Name: 현재차량등록번호, dtype: int64

In [112]:
df2['현재차량등록번호'].value_counts().index

Index(['복합소형', '세라컴(본사공장)', '중형화물'], dtype='object')

In [81]:
df2['현재차량등록번호'].unique()

array([nan, '세라컴(본사공장)', '복합소형', '중형화물'], dtype=object)

In [111]:
error6 = dict()
for one in tqdm(df2['현재차량등록번호'].value_counts().index):
    temp = df2[df2['현재차량등록번호'] == one].index
    error6[one] = temp

100%|██████████| 3/3 [00:00<00:00, 61.34it/s]


### 7\. 차명(제외)

In [78]:
df2['차명'].value_counts()

싼타페(SANTAFE)    71914
포터Ⅱ(PORTERⅡ)    69185
스타렉스(STAREX)    56205
쏘렌토 GLS         26144
스포티지            22870
                ...  
콤비이동봉사차             1
현대에어로퀸버스            1
포터                  1
스타렉스어린이보호차          1
현대5톤덤프청소차           1
Name: 차명, Length: 2764, dtype: int64

In [80]:
for one in df2['차명'].unique():
    print(one)

에어로버스
현대에어로시티540버스(터보)
신광치과진료차
대우고속버스
대우버스
광림대형암롤트럭
한특11.5톤암롤트럭
대우11톤암롤트럭
신성대형암롤트럭
현대5톤트럭-단축
복사덤프카
라이노5톤
현대3.5톤카고트럭
쌍용트랙타
현대3.5톤트럭
현대5톤트럭-장축
마이티II3.5톤(터보)
그랜버드
에어로익스프레스HSX터보
현대뉴에어로고속버스(E)터보
대우14톤장축카고트럭
현대22톤초장축카고트럭(터보)
현대5톤초장축카고트럭
라이노보냉탑차
현대5톤초장축카고트럭(터보)
현대5톤단축카고트럭(터보)
현대4.5톤초장축카고트럭
현대19톤초장축카고트럭(터보)
현대5톤장축카고트럭
현대5톤장축카고트럭(터보)
스카니아트랙터
현대트랙터(터보)
현대11.5톤장축카고트럭(터보)
대우21톤카고트럭
현대슈퍼트럭
현대19.5톤단축카고트럭
대우16톤표준카고트럭
대우8톤장축카고트럭
현대9.5톤초장축저상카고트럭
현대5톤트럭
현대9.5톤초장축카고트럭(터보)
20190624
대우11.5톤카고트럭
한국토미22.5톤카고트럭
현대14톤장축카고트럭(터보)
현대4.5톤초장축카고트럭(터보)
현대4.5톤장축카고트럭(터보)
대우8톤초장축카고트럭
현대9.5톤저상카고트럭(터보)
현대4.5톤트럭
석영사료운반차
에어로스페이스LS터보
현대6KL탱크로리
BH115E
라이노4.5톤
대지20KL탱크로리
현대8톤초장축카고트럭
현대트랙터
한국쓰리축콘테이너트럭
현대25톤초장축카고트럭(터보)
현대16톤단축카고트럭(터보)
한차차량수송트럭
현대11.5톤장축    카고트럭(터보)
20200703
현대20톤카고트럭(터보)
현대16톤중축카고트럭(터보)
현대9.5톤저상카고트럭
현대19.5톤단축카고트럭(터보)
대우22.5톤카고트럭
대양20000L버큠로리
대양11.5톤암롤트럭
현대암롤트럭(터보)
현대20.5톤단축카고트럭(터보)
현대8.5톤장축카고트럭(터보)
현대9.5톤카고트럭
한차차량운송풀트랙터
현대5톤트럭-초장축
BH116
대우트랙터
그랜버드(GRANBIRD)
대우16톤카고트럭
다임19KL액화질소탱크로리
현대5톤장축         카고트럭
현대5톤장축    

In [ ]:
# 20190624, BH115E, 20200703, BH116, 20180911, BS106, BH119, BH117, BH120, BM090, 20190621, BH090, 20191028, 20200521, 20181228, 20200721, 20191004, BS090, 20200225, 중형화물, BX212, 영부환경(주), 

### 8\. 용도

In [82]:
df2['용도'].value_counts()

단거리운행차량        466806
장거리운행차량        131294
기타승합            94150
기타화물            24777
특수차량            10458
150000           8666
건설기계운행차량         6603
청소차량             4953
학원차량             3488
관광버스             2278
회사차량             2105
고속버스             1025
시내버스              283
학교차량              272
시외버스               90
마을버스               48
청소차                23
1종                  5
H191014-179         3
H190905-277         3
XO121772            3
XO121123            3
XO100428            3
H191014-130         3
H191014-128         3
H190511-112         3
XO121657            3
20201005            3
H191014-190         2
XO121658            2
20210804            2
H191014-182         2
20191011            2
H191014-176         2
H190618-221         2
20220818            1
H191014-171         1
20210817            1
20201208            1
H190812-258         1
20210114            1
H190910-064         1
H190724-221         1
20220317            1
XP020528            1
20190711  

In [113]:
p8 = re.compile('[0-9]+')
uni8 = []
for one in df2['용도'].unique():
    try:
        if p8.search(one):
            uni8.append(one)
    except:
        pass

In [114]:
uni8[:5]

['150000', '20190711', '20220317', '20210804', '20201005']

In [119]:
error8 = dict()
for one in tqdm(uni8):
    temp = df2[df2['용도'] == one].index
    error8[one] = temp

100%|██████████| 32/32 [00:01<00:00, 24.21it/s]


### 9\. 차종

In [116]:
df2['차종'].value_counts()

대형승용                       166378
중형화물                       164163
중형승용                       119352
중형승합                        91380
소형화물                        87739
                            ...  
200101-591/200429-636           1
H191201-824/H210827-540         1
180201-149/201028-055           1
1191201-897/211001-0060         1
XO121657/XP020118               1
Name: 차종, Length: 65, dtype: int64

In [117]:
p9 = re.compile('[0-9]+')
uni9 = []
for one in df2['차종'].unique():
    try:
        if p9.search(one):
            uni9.append(one)
    except:
        pass

In [118]:
uni9[:5]

['300000', '150000', '450000', '0', '200801-767/211130-1921']

In [120]:
error9 = dict()
for one in tqdm(uni9):
    temp = df2[df2['차종'] == one].index
    error9[one] = temp

100%|██████████| 46/46 [00:01<00:00, 24.76it/s]


### 10\. 주행거리

In [121]:
df2['주행거리'].value_counts()

보증기간내     5809
보증기간경과    2857
0         2389
174210     174
214210     164
          ... 
181353       1
218813       1
442567       1
418834       1
207438       1
Name: 주행거리, Length: 353618, dtype: int64

### 11\. 저감장치구분

In [122]:
df2['저감장치구분'].value_counts()

1종        744883
1종+SCR      3771
0           1560
1            603
2            380
3            224
4             71
고정식           42
보증기간내         15
5             14
6              5
BQ953          3
AJ6347         2
보증기간경과         1
Name: 저감장치구분, dtype: int64

### 12\. 저감장치종류

In [123]:
df2['저감장치종류'].value_counts()

복합소형             546262
복합중형             109707
자연대형              38377
자연중형              30701
복합대형              19836
                  ...  
BT816/BS066           1
BQ953/BT816           1
BQ953/BQ953           1
AK7958/AL1044         1
AK8133/AK7958         1
Name: 저감장치종류, Length: 103, dtype: int64

### 13\. 인증모델번호

In [124]:
df2['인증모델번호'].value_counts()

BF1-ED-01    102916
AF1-SK-01     95619
BF1-IJ-01     92871
BF1-HK-01     81484
BF1-CE-01     78967
              ...  
8.0               1
4.2               1
1.0               1
0.74              1
4.4               1
Name: 인증모델번호, Length: 105, dtype: int64

### 14\. 장치일련번호

In [125]:
df2['장치일련번호'].value_counts()

60             1683
80             1076
70              819
58              441
100             308
               ... 
BK524             1
AB0800            1
H200427-070       1
BK499             1
D2106567          1
Name: 장치일련번호, Length: 313797, dtype: int64

### 15\. 장치일련번호_전후

In [126]:
df2['장치일련번호_전후'].value_counts()

0                            3895
3                             865
2                             814
1                             777
4                             686
                             ... 
D1900439/D2004975               1
D2004975/D1600323               1
SF-3B-001844/SF-3B-001012       1
SF-3B-001012/SF-3B-001699       1
ILH6455                         1
Name: 장치일련번호_전후, Length: 494219, dtype: int64

### 16\. 클리닝일자

In [127]:
df2['클리닝일자'].value_counts()

20081215    1896
서울시         1486
경기도         1440
20220419    1033
20220516     988
            ... 
20110203       1
20110220       1
20120401       1
AJ0997         1
20220825       1
Name: 클리닝일자, Length: 5030, dtype: int64

### 17\. 클리닝종류

In [128]:
df2['클리닝종류'].value_counts()

고정식         658408
이동식          81491
서구             151
안산시            139
강서구            111
             ...  
20201017         1
20201010         1
20200330         1
속초시              1
20210411         1
Name: 클리닝종류, Length: 172, dtype: int64

### 18\. 제작사코드

In [129]:
df2['제작사코드'].value_counts()

(주)크린어스         20001
N                8666
SK이노베이션(주)       8442
현대모비스(주)         7122
에이치케이엠엔에스(주)     4396
(주)이엔드디          4209
존슨매티카탈리스트코리아     1027
(주)에코닉스           263
후지노테크(주)          127
고정식                78
보증기간내              42
이동식                 7
강서구                 4
시흥시                 3
미추홀구                2
20220203            1
20210618            1
20210226            1
20220422            1
Name: 제작사코드, dtype: int64

### 19\. 클리닝업체명

In [130]:
df2['클리닝업체명'].value_counts()

(주)서천제일자동차공업사    113574
해성                84966
(주)이엔드디(공장)       82441
세라컴(본사공장)         63210
북항자동차정비검사소        51024
                  ...  
201910                3
(주)의정부카독크             3
(주)중산자동차정비검사소         3
광명자동차공업사              3
(주)성우자동차정비            3
Name: 클리닝업체명, Length: 147, dtype: int64

## df로 변경

### values 리스트 길이 맞추기

In [93]:
m = 0
for one in error1.values():
    if m <= len(one):
        m = len(one)
for k, v in error1.items():
    if len(v) != m:
        error1[k] = list(v) + [np.nan]*(m-len(v))

In [95]:
m = 0
for one in error2.values():
    if m <= len(one):
        m = len(one)
for k, v in error2.items():
    if len(v) != m:
        error2[k] = list(v) + [np.nan]*(m-len(v))

In [96]:
m = 0
for one in error3.values():
    if m <= len(one):
        m = len(one)
for k, v in error3.items():
    if len(v) != m:
        error3[k] = list(v) + [np.nan]*(m-len(v))

In [97]:
m = 0
for one in error4.values():
    if m <= len(one):
        m = len(one)
for k, v in error4.items():
    if len(v) != m:
        error4[k] = list(v) + [np.nan]*(m-len(v))

In [98]:
m = 0
for one in error5.values():
    if m <= len(one):
        m = len(one)
for k, v in error5.items():
    if len(v) != m:
        error5[k] = list(v) + [np.nan]*(m-len(v))

In [99]:
edf1 = pd.DataFrame(error1)
edf2 = pd.DataFrame(error2)
edf3 = pd.DataFrame(error3)
edf4 = pd.DataFrame(error4)
edf5 = pd.DataFrame(error5)

In [107]:
edf1.head()

,1톤,승인,(STAREX),/,9,(1%),(SANTAFE),/AS21,-048,/BN937,...,/SF-1-0010578,/AS1076,/AD0242,/AE0085,/AE0528,/SF-1-0010829,/AI1827,/BN064,/AE1546,/BF-L00918
0,454813,5412,547267.0,139710.0,181438.0,370436.0,486450.0,239374.0,370231.0,318433.0,...,268213.0,268177.0,268154.0,268123.0,268121.0,268096.0,268087.0,268022.0,268007.0,757541.0
1,454815,11604,547269.0,252688.0,215352.0,370438.0,486452.0,312446.0,370233.0,338638.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,454817,34973,547271.0,254043.0,351361.0,370440.0,546652.0,341737.0,370235.0,342237.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,512295,48887,551287.0,277508.0,379067.0,370442.0,546654.0,361775.0,370237.0,363325.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,515295,138487,551289.0,278201.0,392444.0,370445.0,546656.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 엑셀로 출력

In [109]:
total_error_file = pd.ExcelWriter('error_indexes.xlsx')
edf1.to_excel(total_error_file, sheet_name='클리닝진행상태')
edf2.to_excel(total_error_file, sheet_name='클리닝기간구분')
edf3.to_excel(total_error_file, sheet_name='차량번호')
edf4.to_excel(total_error_file, sheet_name='차대번호')
edf5.to_excel(total_error_file, sheet_name='구조변경일자')
total_error_file.save()

# 코드마지막